<a href="https://colab.research.google.com/github/suriya43426/Classification_CNN_SugarCrystallization/blob/master/CNN_Master_Project_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import datetime
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>Data_Labiling_Process</b>
    |______ <b>labile</b>: [labile_1.png, labile_2.png, ...,labile_256.png]
    |______ <b>labile</b>: [labile_1.png, labile_2.png, ...,labile_256.png]
    |______ <b>labile</b>: [labile_1.png, labile_2.png, ...,labile_256.png]
    |______ <b>labile</b>: [labile_1.png, labile_2.png, ...,labile_256.png]
</pre>

In [ ]:
!rm -rf ./logs/ 

In [ ]:
%load_ext tensorboard

In [ ]:
pd.set_option("display.precision", 8)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/master_project/labeling_data'

In [ ]:
data_root='/content/drive/My Drive/master_project/labeling_data'

In [ ]:
IMAGE_SHAPE = (244, 244)
TRAINING_DATA_DIR = str(data_root)
print(TRAINING_DATA_DIR);

datagen_kwargs = dict(rescale=1./255, validation_split=.20, rotation_range=45)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

valid_generator = valid_datagen.flow_from_directory(
                                            TRAINING_DATA_DIR,
                                            subset="validation",
                                            shuffle=True,
                                            target_size=IMAGE_SHAPE
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
                                            TRAINING_DATA_DIR,
                                            subset="training",
                                            shuffle=True,
                                            target_size=IMAGE_SHAPE
                                            )

In [ ]:
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

In [ ]:
plt.figure()
plt.imshow(image_batch_train[0])
plt.colorbar()
plt.grid(True)
plt.show()

In [ ]:
model = tf.keras.Sequential([hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                            output_shape=[1280],
                            trainable=False),
                            tf.keras.layers.Dropout(0.4),
                            tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
                            ])

model.build([None, 224, 224, 3])
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='categorical_crossentropy',
                metrics=['acc'])

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
model.fit(
  train_generator,
  epochs=25,
  verbose=1,
  steps_per_epoch=steps_per_epoch,
  validation_data=valid_generator,
  validation_steps=val_steps_per_epoch,
  callbacks=[tensorboard_callback])

In [ ]:
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
model.save("model.h5")

In [ ]:
val_image_batch, val_label_batch = next(iter(valid_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)

print("Validation batch shape:", val_image_batch.shape)

In [ ]:
tf_model_predictions = model.predict(val_image_batch)
print("Prediction results shape:", tf_model_predictions.shape)